# Ch05: Bash Tutorial
___

There are many different shells in our system. Let's have a look:

In [3]:
cat /etc/shells | sed -n -e '1p;2h;2!H;${g;s/\n/;/g;p}'

# /etc/shells: valid login shells
/bin/sh;/bin/dash;/bin/bash;/bin/rbash;/bin/ksh93;/bin/zsh;/usr/bin/zsh;/bin/tcsh;/usr/bin/tcsh;/bin/csh;/usr/bin/screen;/usr/bin/tmux


## Two types of bash shells（两种shells）

1. Non-login shells: `~/.bashrc` as the startup file （非登录shell）
2. Login shells: `/etc/profile`, `~/bash_profile`, `~/.bash_login`, `~/.profile` as the startup files （登录shell）

Note: Here the `rc` is the acronym of __`resource configuration`__.

___on login__:
```bash
if -e /etc/profile; then
    source /etc/profile
end
if -e ~/.bash_profile; then
    source ~/.bash_profile
elif -e ~/.bash_login; then
    source ~/.bash_login
elif -e ~/.profile; then
    source ~/.profile
end
```

Here is a small bash program `sysinfo.sh`:

In [10]:
cat > sysinfo.sh <<EOF
#!/bin/bash 
clear 
echo "This is information provided by my script sysinfo.sh." 
echo "====================================================="
echo "Hello, $USER"
echo 

echo "Today's date is `date`, this is week `date +"%V"`." 
echo

echo "These users are currently connected:" 
w | cut -d " " -f 1 | grep -v USER | sort -u 
echo 

echo "This is `uname -s` running on a `uname -m` processor." 
echo

echo "This is the uptime information:" 
uptime 
echo 

echo "That's all folks."
echo "====================================================="
EOF
chmod u+x sysinfo.sh
./sysinfo.sh

This is information provided by my script sysinfo.sh.
Hello, bio

Today's date is 2016年 03月 03日 星期四 13:09:57 CST, this is week 09.

These users are currently connected:

bio

This is Linux running on a x86_64 processor.

This is the uptime information:
 13:09:57 up 5 days, 23:31,  8 users,  load average: 2.13, 0.93, 0.64

That's all folks.


This kind of execution will create a new shell to run the script.
> If you don’t want to start a new shell but execute the script in the current shell, you `source` it.

## How to define a variable

Here's the __customs for defining a variable__:
* Use upper-case characters (大写字母) for global variable name (全局变量名), while lower-case (小写字母) for local variable (局部变量).
* Use double quotes (双引号) to enclose the value of the variables.

Here is the example:

In [9]:
local name="Jacky Wu"
NAME="Jacky Chan"

By default a variable without a modifier __`local`__ is __global variable__.

#### No space is allowed beside the assignment `=`:

In [11]:
name ="John"

No command 'name' found, did you mean:
 Command 'namei' from package 'util-linux' (main)
 Command 'lame' from package 'lame' (universe)
 Command 'mame' from package 'mame' (multiverse)
 Command 'named' from package 'bind9' (main)
 Command 'nam' from package 'nam' (universe)
 Command 'nama' from package 'nama' (universe)
 Command 'uname' from package 'coreutils' (main)
name: command not found


or

In [12]:
name= "John"

No command 'John' found, did you mean:
 Command 'john' from package 'john' (main)
John: command not found


In [13]:
echo $name

#### Unless we use `export` command, the variable will not be inherited by the child process.

In [4]:
unset bill
cat > test_export.sh <<EOF
#!/bin/zsh
echo "My names is \$bill"
EOF

In [5]:
chmod u+x test_export.sh
unset bill
echo $bill
bill="Clinton"
bash/test_export.sh

My names is 


But if we use `export` like that:

In [6]:
unset bill
export bill="Clinton"
bash/test_export.sh

My names is Clinton


However, the reassignment in the child process will not change the variable in the parent process.

In [7]:
cat > bash/test_export2.sh <<EOF
#!/bin/zsh
bill="Fraser"
echo "My names is \$bill"
EOF

In [8]:
echo $bill

Clinton


In [9]:
bash/test_export2.sh
echo $bill

Clinton


So we can say that:
> The command `export` has only one-way effect which is downward（命令的作用方向是单向且向下的）

## Positional parameters for bash script

### 1. `$*` and `$@`

In [11]:
cat -n bash/test.sh

     1	#!/bin/bash
     2	echo "\$* is $*"
     3	echo "\$@ is $@"


In [14]:
bash/test.sh "Borne shell" "Borne-again shell" "C shell"

$* is Borne shell Borne-again shell C shell
$@ is Borne shell Borne-again shell C shell


It seems that `$*` and `$@` given the same results. But wait, if we use `for` to iterate over all the parameters with or without double quotes, things will be completely different: 

In [22]:
cat -n bash/test2.sh

     1	#!/bin/bash
     2	echo
     3	echo "1. Here is the result for \$*:"
     4	for i in $*; do
     5		echo $i
     6	done
     7	echo
     8	echo "2. Here is the result for \"\$*\":"
     9	for i in "$*"; do
    10		echo $i
    11	done
    12	echo
    13	echo "3. Here is the result for \$@:"
    14	for i in $@; do
    15	    echo $i
    16	done
    17	echo
    18	echo "4. Here is the result for \"\$@\":"
    19	for i in "$@"; do
    20	    echo $i
    21	done


In [23]:
bash/test2.sh "Borne shell" "Borne-again shell" "C shell"


1. Here is the result for $*:
Borne
shell
Borne-again
shell
C
shell

2. Here is the result for "$*":
Borne shell Borne-again shell C shell

3. Here is the result for $@:
Borne
shell
Borne-again
shell
C
shell

4. Here is the result for "$@":
Borne shell
Borne-again shell
C shell


We can see that:
* Without double quotes, `$*` and `$@` are the same;
* With double quotes, `"$*"` will be treated as a whole single string; however, `"$@"` will be treated as a series of parameters, as we expect.

But is this the final answer? Here we use different `IFS` to see if this is true:

In [32]:
cat -n bash/test3.sh

     1	#!/bin/bash
     2	
     3	IFS=" "
     4	echo "By default, IFS is \"$IFS\""
     5	echo -e "\$* is $*"
     6	echo -e "\$@ is $@"
     7	echo -e "\"\$*\" is "$*""
     8	echo -e "\"\$@\" is "$@""
     9	
    10	echo
    11	IFS=":"
    12	echo "But when we change IFS to \"$IFS\""
    13	echo -e "\$* is $*"
    14	echo -e "\$@ is $@"
    15	echo -e "\"\$*\" is "$*""
    16	echo -e "\"\$@\" is "$@""


In [33]:
bash/test3.sh "Borne shell" "Borne-again shell" "C shell"

By default, IFS is " "
$* is Borne shell Borne-again shell C shell
$@ is Borne shell Borne-again shell C shell
"$*" is Borne shell Borne-again shell C shell
"$@" is Borne shell Borne-again shell C shell

But when we change IFS to ":"
$* is Borne shell:Borne-again shell:C shell
$@ is Borne shell Borne-again shell C shell
"$*" is Borne shell Borne-again shell C shell
"$@" is Borne shell Borne-again shell C shell


We could use a table to conclude this section:

| Parameters | Definition |
| :---: | --- |
| `$*` | `"$1 $2 $3..."` |
| `"$*"` | `"$1$IFS[0]$2$IFS[0]$3..."` |
| `$@` | `"$1 $2 $3..."` |
| `"$@"` | `"$1""$2""$3"..."` |

That is:
> *  `$*` expands to the positional parameters, starting from one. When the expansion occurs within double quotes, it expands to a single word with the value of each parameter separated by the __first character of the IFS__ special varible.
> * `$@` expands to the positional parameters, starting from one. When the expansion occurs within double quotes, each parameter expands to a separate word.

#### The other positional parameters

We could summarize the other positional parameters as shown below:

| Positional parameters | Definition |
| :---: | --- |
| `$#` | The number of positional parameters, starting from 1. |
| `$0` | The name of the bash script. |
| `$1`, `$2`, ..., `${10}`, `${11}` | The 1st, 2nd, ..., 10th, 11th positional parameter. |  

## Quotes

### 1. single quotes

Single quotes('') are used to preserve the literal value of each character enclosed within the quotes. A single quote may not occur between single quotes, even when preceded by a backslash.

In [37]:
cat -n bash/test5.sh

     1	#!/bin/bash
     2	today='`date`'
     3	echo -e "When using '\`date\`' (single quotes):"
     4	echo -e "today is: $today"
     5	echo -e 'today is: $today'
     6	echo -e 'today is: \$today'
     7	echo
     8	
     9	today="`date`"
    10	echo -e "When using \"\`date\`\" (Double quotes):"
    11	echo -e "today is: $today"
    12	echo -e 'today is: $today'
    13	echo -e 'today is: \$today'


In [38]:
bash/test5.sh

When using '`date`' (single quotes):
today is: `date`
today is: $today
today is: \$today

When using "`date`" (Double quotes):
today is: 2016年 03月 03日 星期四 14:25:07 CST
today is: $today
today is: \$today


### 2. double quotes

Using double quotes the literal value of all characters enclosed is preserved, except for the dollar sign (`$`), the backquotes (\`) and the backslash (`\`).

The dollar sign and the backquotes retain their special meaning within the double quotes.

The backslash retains its meaning only when followed by dollar, backquote, double quote, backslash or newline. Within double quotes, the backslashes are removed from the input stream when followed by one of these characters. Backslashes preceding characters that don’t have a special meaning are left unmodified for processing by the shell interpreter.

## Shell expansions

### 1. brace expansion

In [40]:
echo "sp"{an,ort,ill}

span sport spill


In [41]:
echo "Michael "{Johndon,Smith,Jackson}

Michael Johndon Michael Smith Michael Jackson


In [46]:
echo "Michael "{Johndon, Smith, Jackson}

Michael {Johndon, Smith, Jackson}


We could see that
> It is treated as a whole string when spaces are put before or after the colons within the brace.

### 2. tilde expansion

| Tilde parameter | Definition | Example |
| :---: | --- | --- |
| `~` | Home directory of the current user | `cd ~` |
| `~USER` | Home directory of `USER` | `cd ~root` |
| `~+` | The next working directory. | `cd ~+` |
| `~-` | The previous working directory. | `cd ~-` |

Here we could also use `pushd` to push directory into the __directory stack__, and then use __`cd ~-1`, `cd ~+1`, or `cd ~+n`, `cd ~-n`__ to leap between the directories stored in the stack.

### 3. parameter expansion and variable expansion

In parameter and variable expansion, the dollar sign (`$`), and the curly brace (`{}`) play an essential role. 

#### (1) parameter expansion

For positional parameters, when the position is over 10, we should use the brace to avoid confusion. Here is an example to output the 10th positional parameter:

In [58]:
cat bash/test7a.sh

#!/bin/bash
echo $10


In [60]:
bash/test7a.sh 0 1 2 3 4 5 6 7 8 9

00


To avoid the obscure definition, we should use brace here:

In [61]:
cat bash/test7b.sh

#!/bin/bash
echo ${10}


In [62]:
bash/test7b.sh 0 1 2 3 4 5 6 7 8 9

9


#### (2) direct variable expansion

Direct variable expansion use the value of the variable to substitute the string. Here is an example:

In [67]:
cat -n bash/test8a.sh

     1	#!/bin/bash
     2	firstname="Wang"
     3	echo -e "The poet is named ${firstname}guozhen"


In [68]:
bash/test8a.sh

The poet is named Wangguozhen


#### (3) Indirect variable expansion

In indirect variable expansion, the variable name is also a variable. And then we can use `!` to expand the variable name. Here is an example. 

In [69]:
cat -n bash/test8b.sh

     1	#!/bin/bash
     2	firstname="Wang"
     3	first="firstname"
     4	echo -e "The poet is named ${!first}guozhen"


In [70]:
bash/test8b.sh

The poet is named Wangguozhen


### 4. command substitution

It is to note that we can use __`$(COMMAND)`__ to do command substitution instead of command execution. Here is an example: 

In [71]:
cat bash/test9.sh

#!/bin/bash
# command substitution

echo
date

echo
$(date)


In [73]:
bash -x bash/test9.sh

+ echo

+ date
2016年 03月 03日 星期四 15:46:25 CST
+ echo

++ date
+ 2016年 03月 03日 星期四 15:46:25 CST
bash/test9.sh: line 8: 2016年: command not found


In this example the output for the command `date` is treated as a series of command. But unfortunately, the command does not exist.

The second type of command substitution uses the backquotes (__\`COMMAND\`__).

### 5. arithmetic expansion

To expand the arithmetic expression, you need to use __`$((EXPRESSION))`__.
> All tokens in the expression undergo parameter expansion, command substitution, and quote removal.

Here we use an example to illustrate the usage of arithmetic expansion:

In [74]:
cat bash/test10.sh

#!/bin/bash
echo

val=3

echo $(($1+val+2+`date +%d`))


In [8]:
bash -x bash/test10.sh

+ echo

+ val=3
++ date +%d
+ echo 24
24


In [9]:
bash -x bash/test10.sh 2

+ echo

+ val=3
++ date +%d
+ echo 26
26


Here we show another example to illustrate the conversion between binary number, octal number, hexadecimal number, and decimal number. 

In [77]:
cat bash/test11.sh

#!/bin/bash

echo "octal numbers" 
echo "011==$((011))" 

echo 
echo "hexadecimal numbers" 
echo "0x1a==$((0x1a))" 

echo
echo "binary numbers"
echo "01110==$((2#01110))"

echo 
echo "arithmetic base 36" 
echo "36#z==$((36#z))" 
echo "36#Z==$((36#Z))" 

echo 
echo "arithmetic base 37" 
echo "37#a==$((37#a))" 
echo "37#A==$((37#A))" 

echo 
echo "arithmetic base 64" 
echo "64#@==$((64#@))" 
echo "64#_==$((64#_))"


In [79]:
bash -x bash/test11.sh

+ echo 'octal numbers'
octal numbers
+ echo 011==9
011==9
+ echo

+ echo 'hexadecimal numbers'
hexadecimal numbers
+ echo 0x1a==26
0x1a==26
+ echo

+ echo 'binary numbers'
binary numbers
+ echo 01110==14
01110==14
+ echo

+ echo 'arithmetic base 36'
arithmetic base 36
+ echo 36#z==35
36#z==35
+ echo 36#Z==35
36#Z==35
+ echo

+ echo 'arithmetic base 37'
arithmetic base 37
+ echo 37#a==10
37#a==10
+ echo 37#A==36
37#A==36
+ echo

+ echo 'arithmetic base 64'
arithmetic base 64
+ echo 64#@==62
64#@==62
+ echo 64#_==63
64#_==63


### 6. process expansion

In [1]:
ls -l <(true)

lr-x------ 1 bio bio 64  3月  3 16:11 /dev/fd/63 -> pipe:[5287275]


The process of running the command `true` results in a pipe file named `/dev/fd/63` where `63` is the file descriptor for this file. 

* If the ">(LIST)" form is used, writing to the file will provide input for LIST.
* If the "<(LIST)" form is used, the file passed as an argument should be read to obtain the output of LIST.

In [2]:
ls -l >(false)

l-wx------ 1 bio bio 64  3月  3 16:16 /dev/fd/63 -> pipe:[5290761]


### 7. word splitting

In [5]:
cat -n bash/test12.sh

     1	#!/bin/bash
     2	filename="path1/path2/file.ext"
     3	ext=${filename#*.}
     4	basename=${filename##*/}
     5	toppath=${filename%%/*}
     6	comppath=${filename%/*}
     7	echo -e "Extension: $ext"
     8	echo -e "Basename: $basename"
     9	echo -e "Top path: $toppath"
    10	echo -e "Complete path: $comppath"


In [8]:
bash -x bash/test12.sh

bash -x bash/test12.sh
+ bash -x bash/test12.sh
+ filename=path1/path2/file.ext
+ ext=ext
+ basename=file.ext
+ toppath=path1
+ comppath=path1/path2
+ echo -e 'Extension: ext'
Extension: ext
+ echo -e 'Basename: file.ext'
Basename: file.ext
+ echo -e 'Top path: path1'
Top path: path1
+ echo -e 'Complete path: path1/path2'
Complete path: path1/path2


### 8. default value expansion

| Expression1 | Expression2 | Definition |
| :---: | :---: | --- |
| `echo ${var-6}` | `echo ${var:-6}` | print 6 if `var` not set; otherwise print it. |
| `echo ${var+6}` | `echo ${var:=6}` | print 6 if `var` set; otherwise print null. |
| `echo ${var=6}` | `echo ${var:+6}` | set `var=6` and print it if `var` not set; otherwise only print it. | 

In [21]:
unset undefined
echo ${undefined:-5}

echo ${undefined:-5}
+ echo 5
5


In [22]:
echo $undefined

echo $undefined
+ echo



In [23]:
echo ${undefined:-10}

echo ${undefined:-10}
+ echo 10
10


In [24]:
echo $undefined

echo $undefined
+ echo



In [13]:
unset undefined
echo ${undefined=5}

echo ${undefined=5}
+ echo 5
5


In [14]:
echo $undefined

echo $undefined
+ echo 5
5


In [15]:
echo ${undefined+10}

echo ${undefined+10}
+ echo 10
10


In [16]:
echo $undefined

echo $undefined
+ echo 5
5


In [17]:
echo ${undefined=10}

echo ${undefined=10}
+ echo 5
5


In [6]:
unset undefined
echo ${undefined=10}

10


In [7]:
echo $undefined

10


In [4]:
unset undefined
echo ${undefined+6}

In [3]:
undefined=5
echo ${undefined+6}

6


In [5]:
echo ${undefined}

In [19]:
echo $undefined

echo $undefined
+ echo



In [30]:
if test -n undefined; then echo true; else echo false; fi

if test -n undefined; then echo true; else echo false; fi
+ test -n undefined
+ echo true
true


However, when the variable is a `null` string, things can be very different:

In [23]:
unset undefined
undefined=
echo ${undefined:-5}

5


In [24]:
echo $undefined

In [11]:
echo ${undefined:=5}

5


In [12]:
echo $undefined

5


In [13]:
unset undefined
undefined=
echo ${undefined:+5}

## Conditional expressions
* [[ __compound command__ ]]
* [ __built-in command__ ]
* `test` built-in command

In [43]:
if [[ -s test ]]; then
    echo "test is a symbolic link"
else
    echo "test is not a symbolic link"
fi

test is a symbolic link


### Unary operators
| Unary operator | Description |
| :---: | --- |
| `[ -a FILE ]` | True if FILE is __available__. |
| `[ -b FILE ]` | True if FILE exists and is a __block-special__ file. |
| `[ -c FILE ]` | True if FILE exists and is a __character-special__ file.|
| `[ -d FILE ]` | True if FILE exists and is a __directory__. |
| `[ -e FILE ]` | True if FILE __exists__. |
| `[ -f FILE ]` | True if FILE exists and is a __regular__ file. |
| `[ -g FILE ]` | True if FILE exists and its __SGID__ bit is set. |
| `[ -h FILE ]` | True if FILE exists and is a __symbolic link__. |
| `[ -k FILE ]` | True if FILE exists and its __sticky bit__ is set. |
| `[ -p FILE ]` | True if FILE exists and is a __named pipe (FIFO)__. |
| `[ -r FILE ]` | True if FILE exists and is __readable__. |
| `[ -s FILE ]` | True if FILE exists and has a __non-zero size__. |
| `[ -t FD ]`   | True if file descriptor (FD) is open and refers to a __terminal__. |
| `[ -u FILE ]` | True if FILE exists and its __SUID__ bit is set. |
| `[ -w FILE ]` | True if FILE exists and is __writable__. |
| `[ -x FILE ]` | True if FILE exists and is __executable__. |
| `[ -O FILE ]` | True if FILE exists and is __owned by the EUID__.|
| `[ -G FILE ]` | True if FILE exists and is __owned by the EGID__.|
| `[ -L FILE ]` | True if FILE exists and is a __symbolic link__. |
| `[ -N FILE ]` | True if FILE exists and has been __modified__ since it was last read. |
| `[ -S FILE ]` | True if FILE exists and is a __socket__. |
| `[ -o OPTIONNAME ]` | True if shell option "OPTIONNAME" is enabled. |
| `[ -z STRING ]` | True if "STRING" is empty. |
| `[ -n STRING ]` or `[ STRING ]` | True if "STRING" is non-empty. |

### binary operators
| Binary operator | Description |
| :---: | --- |
| `[ FILE1 -nt FILE2 ]` | True if FILE1 is newer than FILE2, or if FILE1 exists and FILE2 does not. |
| `[ FILE1 -ot FILE2 ]` | True if FILE1 is older than FILE2, or is FILE2 exists and FILE1 does not. |
| `[ FILE1 -ef FILE2 ]` | True if FILE1 and FILE2 refer to the same device and inode numbers. |
| `[ STRING1 == STRING2 ]` | True if the strings are equal. "=" may be used instead of "==" for strict POSIX compliance. |
| `[ STRING1 != STRING2 ]` | True if the strings are not equal. |
| `[ STRING1 < STRING2 ]` | True if "STRING1" sorts before "STRING2" lexicographically. |
| `[ STRING1 > STRING2 ]` | True if "STRING1" sorts after "STRING2" lexicographically. |
| `[ ARG1 OP ARG2 ]` | "OP" is one of -eq, -ne, -lt, -le, -gt or -ge. These arithmetic binary operators return true if "ARG1" is equal to, not equal to, less than, less than or equal to, greater than, or greater than or equal to "ARG2", respectively. "ARG1" and "ARG2" are integers. |

### Combining expressions

| Operation | Effect |
| :---: | --- |
| `[ ! EXPR ]` | True if EXPR is false. |
| `[ ( EXPR ) ]` | Returns the value of EXPR. This may be used to override the normal precedence of operators. |
| `[ EXPR1 -a EXPR2 ]` | True if both EXPR1 and EXPR2 are true. |
| `[ EXPR1 -o EXPR2 ]` | True if either EXPR1 or EXPR2 is true. |

In [46]:
grep "^bio2:" /etc/passwd 1>/dev/null 2>&1
if [ $? -ne 0 ]; then
    echo "bio2 is not a valid user on this machine"
fi

bio2 is not a valid user on this machine


In [2]:
if grep "^bio2:" /etc/passwd 1>/dev/null 2>&1; then
    echo "bio2 is not a valid user on this machine"
fi

In [30]:
ls -l test

lrwxrwxrwx 1 bio bio 7  3月  2 15:41 test -> re/test


In [68]:
x="string match"
if [[ "$x" = string* ]]; then
    echo "matching exactly"
fi

matching exactly


__How about changing the above conditional statement as following?__
* `[[ $x == "string*" ]]`
* `[ $x == string* ]`
* `[ "$x" == string* ]`
* `test "$x" == string*`

## `if` statement 

In [70]:
if [[ -f test ]]; then
    echo "test is a regular file"
elif [[ -r test ]]; then
    echo "test is readable"
elif [[ -w test ]]; then
    echo "test is writable"
elif [[ -x test ]]; then
    echo "test is executable"
else
    echo "test is not available"
fi

test is a regular file


## `while` statement

In [23]:
i=5
while [[ $i -lt 10 -a $i -ge 5 ]]; do
    echo $i
    (( i++ ))
done

5
6
7
8
9


## `for` statement

With respect to the `for` statement, there are two types of syntaxes:
#### [1] `for ... in ...`
```bash
for i in `seq 10`; do
    echo $i
done
```

#### [2] C-style
```bash
for ((i=0; i<10; i++)); do
    echo $i
done
```

## `until` expression

In [2]:
x=1
until [[ x -ge 5 ]]; do
    echo $x
    (( x++ ))
done

1
2
3
4


## `select` expression

In [73]:
select item in "ls" "uname -r" "pwd" "ls -l" "break"; do
    `echo $item`
done

1) ls
2) uname -r
3) pwd
4) ls -l
#? 


## `case` expression

In [ ]:
echo "Write down your option: "
read -p "1) ls\n2) uname -r\n3) pwd\n4) ls -l\n5) break" opt
case $opt in
1) ls ;;
2) uname -r ;;
3) pwd ;;
4) ls -l ;;
5) break ;;
esac

1) ls\n2) uname -r\n3) pwd\n4) ls -l\n5) break


In [2]:
echo $PPID

16599


In [5]:
echo $HISTSIZE

500


## `declare`

### 1. declare a read-only variable

In [1]:
unset x
declare -r x=3           # read only
x=5

bash: x: readonly variable


In [2]:
echo $x

3


### 2. declare an integer variable

In [3]:
unset y
declare -i y
y=5/2

In [4]:
echo $y

2


### 3. declare an indexed array

In [5]:
unset z
declare -a z
z=(a b c d e)

In [6]:
echo ${z[1]}

b


In [7]:
for i in ${z[@]}; do echo $i; done

a
b
c
d
e


### 4. declare an associated array 

In [10]:
unset w
declare -A w
w['a']=12
w['b']=13
for i in a b c; do let "w[$i]-=1"; done

In [11]:
for count in ${w[@]}; do echo $count; done

11
12
-1


## Arithmetic computation

### 1. (( ))

In [12]:
unset a
a=1
(( a++ ))
echo $a

2


### 2. `expr`

In [13]:
expr 3 + 5

8


In [14]:
expr 3+5

3+5


In [15]:
expr 3.5 + 5

expr: non-integer argument


In [27]:
expr 3 * 5

expr: syntax error


In [28]:
expr 3 \* 5

15


In [30]:
expr 13 / 5

2


In [31]:
expr 13 % 5

3


### 3. `let`

In [17]:
let "a = $a + 2"
echo $a

4


In [19]:
let "a = $a * 2.5"
echo $a

bash: let: a = 8 * 2.5: syntax error: invalid arithmetic operator (error token is ".5")
8


In [20]:
echo $?

0


### 4. `bc`

In [22]:
echo "3.53*4+2" | bc

16.12


## String operations

### 1. String substitution

In [25]:
var="ACTGCTG"
echo ${var/ACTG/actg}

actgCTG


### 2. substring

In [32]:
echo ${var:2}

TGCTG


In [33]:
echo ${var:3:2}

GC


### 3. string length

In [34]:
echo ${#var}

7


### 4. string truncation

In [40]:
echo "original string: ${var}"

original string: ACTGCTG


In [39]:
echo "Lazy head truncation: ${var#*T}"

Lazy head truncation: GCTG


In [38]:
echo "Greedy head truncation: ${var##*T}"

Greedy head truncation: G


In [41]:
echo "lazy tail truncation: ${var%T*}"

lazy tail truncation: ACTGC


In [42]:
echo "Greedy tail truncation: ${var%%T*}"

Greedy tail truncation: AC


## Random Number

In [43]:
echo $RANDOM

26250


In [50]:
for i in {1..100}
do
    expr 50 + $RANDOM % 50
done > random.txt

In [51]:
head random.txt

79
54
97
78
61
75
64
82
92
51
